In [1]:
%pip install torch --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install "trl>=0.20.0" "peft>=0.17.0" "transformers>=4.55.0" trackio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 332.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 248.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 245.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.0/837.0 kB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 166.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 212.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.8/789.8 kB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 207.0 MB/s eta 0:00:00
  Attempting uninstall: fsspecm╺━━━━━━━━━━━━━━━━━━━━━━━━━ 11/31 [hf-xet]]
    Found existing installation: fsspec 2025.7.0━━━━━━━━━━━━━━ 11/31 [hf-xet]
    Uninstalling fsspec-2025.7.0:━━━━━━━━━

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'],
    num_rows: 1000
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are an AI chatbot with a lively and energetic personality.<|end|><|start|>user<|message|>Can you show me the latest trends on Twitter right now?<|end|><|start|>assistant<|channel|>analysis<|message|>D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.

Je devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque ré

In [5]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

In [6]:
messages = [
    {"role": "user", "content": "¿Cuál es el capital de Australia?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, max_new_tokens=512)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>¿Cuál es el capital de Australia?<|end|><|start|>assistant<|channel|>analysis<|message|>The user asks: "¿Cuál es el capital de Australia?" They want the capital of Australia. The answer: Canberra. Provide Spanish answer.<|end|><|start|>assistant<|channel|>final<|message|>La capital de Australia es **Canberra**.<|return|>


In [7]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 15,040,512 || all params: 20,929,797,696 || trainable%: 0.0719


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


In [8]:
from trl import SFTConfig

training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir="gpt-oss-20b-multilingual-reasoner",
    report_to="trackio",
    push_to_hub=True,
)

In [9]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)
trainer.train()

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

* Trackio project initialized: huggingface
* Trackio metrics logged to: /teamspace/studios/this_studio/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")


Step,Training Loss
1,1.961100
2,2.039200
3,1.806100
4,1.825500
5,1.605400
6,1.564600
7,1.405700
8,1.411000
9,1.213900
10,1.319400


TrainOutput(global_step=63, training_loss=1.1400182190395536, metrics={'train_runtime': 1068.2475, 'train_samples_per_second': 0.936, 'train_steps_per_second': 0.059, 'total_flos': 1.9951547028611328e+17, 'train_loss': 1.1400182190395536})

In [10]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name="HuggingFaceH4/Multilingual-Thinking-finetuned")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ilingual-reasoner/training_args.bin: 100%|##########| 6.16kB / 6.16kB            

  ...ultilingual-reasoner/tokenizer.json: 100%|##########| 27.9MB / 27.9MB            

  ...-reasoner/adapter_model.safetensors: 100%|##########| 60.2MB / 60.2MB            

...

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ilingual-reasoner/training_args.bin: 100%|##########| 6.16kB / 6.16kB            

  ...ultilingual-reasoner/tokenizer.json: 100%|##########| 27.9MB / 27.9MB            

  ...-reasoner/adapter_model.safetensors: 100%|##########| 60.2MB / 60.2MB            

CommitInfo(commit_url='https://huggingface.co/Sandipan1976/gpt-oss-20b-multilingual-reasoner/commit/65a759cb097e718b5b6ef994734e5601a663cbb1', commit_message='End of training', commit_description='', oid='65a759cb097e718b5b6ef994734e5601a663cbb1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sandipan1976/gpt-oss-20b-multilingual-reasoner', endpoint='https://huggingface.co', repo_type='model', repo_id='Sandipan1976/gpt-oss-20b-multilingual-reasoner'), pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
model_kwargs = dict(attn_implementation="eager", torch_dtype="auto", use_cache=True, device_map="auto")
base_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()

peft_model_id = "gpt-oss-20b-multilingual-reasoner"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model = model.merge_and_unload()

MXFP4 quantization requires triton >= 3.4.0 and triton_kernels installed, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


In [2]:
REASONING_LANGUAGE = "Italian"
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "¿Cuál es el capital de India?"  

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.6, "top_p": None, "top_k": None}

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

reasoning language: Italian<|end|><|start|>user<|message|>¿Cuál es el capital de India?<|end|><|start|>assistant<|channel|>analysis<|message|>Okay, l'utente chiede la capitale dell'India. Prima di tutto, devo ricordarmi che la capitale dell'India è New Delhi. Tuttavia, devo assicurarmi che l'utente capisca che la risposta è corretta. Inizierò confermando che New Delhi è effettivamente la capitale dell'India. 

Inoltre, l'utente ha usato lo spagnolo, quindi devo rispondere in spagnolo. Devo assicurarmi di usare una struttura chiara e concisa. Inizierò con una frase che risponda direttamente alla domanda. Poi aggiungerò forse qualche dettaglio in più, come la distinzione tra la capitale amminist

In [3]:
REASONING_LANGUAGE = "Hindi"  
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "What is the national symbol of India?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

reasoning language: Hindi<|end|><|start|>user<|message|>What is the national symbol of India?<|end|><|start|>assistant<|channel|>analysis<|message|>
मुझे लगता है कि उपयोगकर्ता भारत के राष्ट्रीय प्रतीक के बारे में पूछ रहे हैं। चलिए इसे चरण-दर-चरण तोड़ते हैं।

पहले, मुझे यह सुनिश्चित करना चाहिए कि मैं सही प्रतीक से निपट रहा हूँ। भारत के कई राष्ट्रीय प्रतीक हैं, जैसे ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय ध्वज, राष्ट्रीय